In [205]:
import torch

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
from torch import nn, optim
from pandas import Series
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
#register_matplotlib_converters()

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [82]:
df=pd.read_csv("../data/Corona_NLP_train.csv",encoding='ISO-8859-1')

In [83]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [135]:
len(df['TweetAt'].value_counts())

30

In [122]:
df1=df[df["Location"].isin(["United States","New York, NY","Washington, DC","Los Angeles, CA","USA","Chicago, IL","California, USA","San Francisco, CA","Boston, MA","New York, USA","Atlanta, GA","New York"])]

In [126]:
df1['TweetAt']=pd.to_datetime(df1['TweetAt'], format="%d-%m-%Y")

/var/folders/x9/c91lstn973v4tldv_64cr19c0000gn/T/ipykernel_1120/1666013083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['TweetAt']=pd.to_datetime(df1['TweetAt'], format="%d-%m-%Y")


In [129]:
df1=df1[['TweetAt','Sentiment']]

In [217]:
df2=df1["Sentiment"].groupby(df1['TweetAt'].dt.date).apply(lambda x: x.mode()).reset_index()

In [222]:
df2 = df2.drop(df2.columns[[1]], axis=1)


In [227]:
df2

,TweetAt,Sentiment
0,2020-03-16,Positive
1,2020-03-17,Positive
2,2020-03-18,Negative
3,2020-03-19,Positive
4,2020-03-20,Positive
5,2020-03-21,Positive
6,2020-03-22,Positive
7,2020-03-23,Positive
8,2020-03-24,Positive
9,2020-03-25,Positive


In [225]:
df_old=pd.read_csv('../data/time_series_covid19_confirmed_US.csv')

In [232]:
df_old.loc[:, '3/16/20':'4/14/20']

,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,...,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20
0,0,0,0,0,0,0,0,0,1,5,...,12,12,12,12,17,18,19,19,19,23
1,1,1,1,1,2,2,2,3,5,5,...,39,43,47,54,63,64,70,75,82,93
2,0,0,0,0,0,0,0,0,0,0,...,3,4,4,4,8,9,10,10,10,11
3,0,0,0,0,0,0,0,0,0,0,...,7,7,8,9,11,11,13,16,17,18
4,0,0,0,0,0,0,0,0,0,3,...,13,13,13,13,15,15,15,17,18,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,0,0,0,1,2,2,2,2,2,5,...,39,40,41,44,45,50,53,56,56,57
3338,0,0,0,0,0,0,0,0,0,0,...,2,3,3,3,3,4,4,4,4,4
3339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3340,0,0,0,0,0,0,0,0,0,0,...,4,4,4,5,5,5,5,5,5,5
